In [4]:
import ee
import geemap
import pandas as pd
import numpy as np
from datetime import datetime

# Initialize Earth Engine
ee.Initialize()

# Define time range
start_date = '2002-01-01'
end_date = '2013-12-31'

# Define area of interest using the bounds from your stations
aoi = ee.Geometry.Rectangle([84.89, 26.47, 86.96, 27.19])

# Import GRACE data
grace = ee.ImageCollection('NASA/GRACE/MASS_GRIDS/MASCON_CRI') \
    .select('lwe_thickness') \
    .filterDate(start_date, end_date) \
    .filterBounds(aoi)

# Create grid points
grid_points_list = []
for lon in np.arange(84.5, 87.5, 0.5):
    for lat in np.arange(26.0, 27.5, 0.5):
        if (84.89 <= lon <= 86.96) and (26.47 <= lat <= 27.19):
            grid_points_list.append(ee.Feature(
                ee.Geometry.Point([lon, lat]),
                {'longitude': lon, 'latitude': lat}
            ))
grid_points = ee.FeatureCollection(grid_points_list)

# Import station points
stations = ee.FeatureCollection('projects/ee-074bce151saurav/assets/station_coordinates')

def extract_data(image):
    date = ee.Date(image.get('system:time_start'))
    
    # Extract for grid points
    grid_extract = image.reduceRegions(
        collection=grid_points,
        reducer=ee.Reducer.mean().setOutputs(['lwe_thickness']),
        scale=25000
    ).map(lambda f: f.set({
        'year': date.get('year'),
        'month': date.get('month'),
        'type': 'grid'
    }))
    
    # Extract for station points with geometry coordinates
    station_extract = image.reduceRegions(
        collection=stations,
        reducer=ee.Reducer.mean().setOutputs(['lwe_thickness']),
        scale=25000
    ).map(lambda f: ee.Feature(f).set({
        'year': date.get('year'),
        'month': date.get('month'),
        'type': 'station',
        'district': f.get('district'),
        'location': f.get('location'),
        # Extract coordinates from geometry
        'longitude': ee.Feature(f).geometry().coordinates().get(0),
        'latitude': ee.Feature(f).geometry().coordinates().get(1)
    }))
    
    return grid_extract.merge(station_extract)

# Apply extraction to all images
all_data = grace.map(extract_data).flatten()

# Export grid points data
grid_task = ee.batch.Export.table.toDrive(
    collection=all_data.filter(ee.Filter.eq('type', 'grid')),
    description='GRACE_grid_data',
    fileFormat='CSV',
    folder='GRACE_Data',
    selectors=['year', 'month', 'longitude', 'latitude', 'lwe_thickness']
)

# Export station points data
station_task = ee.batch.Export.table.toDrive(
    collection=all_data.filter(ee.Filter.eq('type', 'station')),
    description='GRACE_station_data',
    fileFormat='CSV',
    folder='GRACE_Data',
    selectors=['year', 'month', 'district', 'location', 'longitude', 'latitude', 'lwe_thickness']
)

# Start the export tasks
grid_task.start()
station_task.start()

# Print task information
print("Started export tasks. Please check your Google Drive folder 'GRACE_Data' for the results.")
print("Number of grid points:", grid_points.size().getInfo())
print("Number of stations:", stations.size().getInfo())

Started export tasks. Please check your Google Drive folder 'GRACE_Data' for the results.
Number of grid points: 8
Number of stations: 134


In [9]:
import ee
import geemap
from datetime import datetime
import os

class GEEDataCollector:
    def __init__(self, study_area_bounds):
        """Initialize GEE and set up study area"""
        ee.Initialize()
        self.roi = ee.Geometry.Rectangle(study_area_bounds)
        self.start_date = '2002-01-01'
        self.end_date = '2013-12-31'
        self.setup_directories()

    def setup_directories(self):
        """Create directory structure"""
        base_dir = 'GRACE_downscaling_data'
        subdirs = ['dem', 'landcover', 'soil_moisture', 'precipitation', 'grace', 'processed']
        for subdir in subdirs:
            os.makedirs(os.path.join(base_dir, subdir), exist_ok=True)

    def download_terrain(self):
        """Download SRTM DEM and derivatives"""
        print("Downloading terrain data...")
        srtm = ee.Image('USGS/SRTMGL1_003').clip(self.roi)
        slope = ee.Terrain.slope(srtm)
        aspect = ee.Terrain.aspect(srtm)
        terrain = srtm.addBands(slope).addBands(aspect)
        
        task = ee.batch.Export.image.toDrive(
            image=terrain,
            description='terrain_features',
            folder='GRACE_downscaling',
            scale=10000,
            crs='EPSG:4326',
            region=self.roi,
            maxPixels=1e13
        )
        task.start()
        return task

    def download_landcover(self):
        """Download ESA WorldCover data"""
        print("Downloading land cover data...")
        landcover = ee.ImageCollection("ESA/WorldCover/v100").first().clip(self.roi)
        
        task = ee.batch.Export.image.toDrive(
            image=landcover,
            description='landcover',
            folder='GRACE_downscaling',
            scale=10000,
            crs='EPSG:4326',
            region=self.roi,
            maxPixels=1e13
        )
        task.start()
        return task

    def generate_date_pairs(self):
        """Generate list of year-month pairs for the study period"""
        start = datetime.strptime(self.start_date, '%Y-%m-%d')
        end = datetime.strptime(self.end_date, '%Y-%m-%d')
        
        date_pairs = []
        current = start
        
        while current <= end:
            date_pairs.append((current.year, current.month))
            if current.month == 12:
                current = datetime(current.year + 1, 1, 1)
            else:
                current = datetime(current.year, current.month + 1, 1)
                
        return date_pairs

    def download_soil_moisture(self):
        """Download ERA5-Land soil moisture data"""
        print("Downloading soil moisture data...")
        tasks = []
        
        # Get list of year-month pairs
        date_pairs = self.generate_date_pairs()
        
        for year, month in date_pairs:
            print(f"Processing soil moisture for {year}-{month:02d}")
            
            start_date = ee.Date.fromYMD(year, month, 1)
            end_date = start_date.advance(1, 'month')
            
            # Get soil moisture data for the month
            soil_moisture = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
                .select(['volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2']) \
                .filterDate(start_date, end_date) \
                .mean() \
                .clip(self.roi)
            
            # Create export task
            task = ee.batch.Export.image.toDrive(
                image=soil_moisture,
                description=f'soil_moisture_{year}_{month:02d}',
                folder='GRACE_downscaling',
                scale=10000,
                crs='EPSG:4326',
                region=self.roi,
                maxPixels=1e13
            )
            task.start()
            tasks.append(task)
            
        return tasks

    def download_precipitation(self):
        """Download CHIRPS precipitation data"""
        print("Downloading precipitation data...")
        tasks = []
        
        # Get list of year-month pairs
        date_pairs = self.generate_date_pairs()
        
        for year, month in date_pairs:
            print(f"Processing precipitation for {year}-{month:02d}")
            
            start_date = ee.Date.fromYMD(year, month, 1)
            end_date = start_date.advance(1, 'month')
            
            # Get precipitation data for the month
            precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
                .filterDate(start_date, end_date) \
                .sum() \
                .clip(self.roi)
            
            # Create export task
            task = ee.batch.Export.image.toDrive(
                image=precip,
                description=f'precipitation_{year}_{month:02d}',
                folder='GRACE_downscaling',
                scale=10000,
                crs='EPSG:4326',
                region=self.roi,
                maxPixels=1e13
            )
            task.start()
            tasks.append(task)
            
        return tasks

    def download_all_data(self):
        """Download all datasets"""
        tasks = []
        
        # Download terrain data
        tasks.append(self.download_terrain())
        
        # Download land cover
        tasks.append(self.download_landcover())
        
        # Download soil moisture
        tasks.extend(self.download_soil_moisture())
        
        # Download precipitation
        tasks.extend(self.download_precipitation())
        
        return tasks

def main():
    # Study area bounds [xmin, ymin, xmax, ymax]
    bounds = [84.89, 26.47, 86.96, 27.19]
    
    # Initialize collector and download data
    collector = GEEDataCollector(bounds)
    tasks = collector.download_all_data()
    
    print(f"\nInitiated {len(tasks)} export tasks.")
    print("Files will be exported to 'GRACE_downscaling' folder in your Google Drive.")
    print("Note: The export process may take some time to complete.")

if __name__ == "__main__":
    main()

Processing soil moisture for 2002-01
Processing soil moisture for 2002-02
Processing soil moisture for 2002-03
Processing soil moisture for 2002-04
Processing soil moisture for 2002-05
Processing soil moisture for 2002-06
Processing soil moisture for 2002-07
Processing soil moisture for 2002-08
Processing soil moisture for 2002-09
Processing soil moisture for 2002-10
Processing soil moisture for 2002-11
Processing soil moisture for 2002-12
Processing soil moisture for 2003-01
Processing soil moisture for 2003-02
Processing soil moisture for 2003-03
Processing soil moisture for 2003-04
Processing soil moisture for 2003-05
Processing soil moisture for 2003-06
Processing soil moisture for 2003-07
Processing soil moisture for 2003-08
Processing soil moisture for 2003-09
Processing soil moisture for 2003-10
Processing soil moisture for 2003-11
Processing soil moisture for 2003-12
Processing soil moisture for 2004-01
Processing soil moisture for 2004-02
Processing soil moisture for 2004-03
P

In [14]:
import ee
import geemap
from datetime import datetime
import os
import math

class AdditionalDataCollector:
    def __init__(self, study_area_bounds):
        """Initialize GEE and set up study area"""
        ee.Initialize()
        self.roi = ee.Geometry.Rectangle(study_area_bounds)
        self.start_date = '2002-01-01'
        self.end_date = '2013-12-31'

    def download_terrain_data(self):
        """Download ALOS DEM and calculate terrain indices"""
        print("Processing terrain data...")
        
        # Get ALOS DEM
        dem = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").select('DSM').mean()
        
        # Calculate terrain indices
        slope = ee.Terrain.slope(dem)
        aspect = ee.Terrain.aspect(dem)
        hillshade = ee.Terrain.hillshade(dem)
        
        # Combine all terrain features
        terrain = dem.addBands(slope).addBands(aspect).addBands(hillshade)
        terrain = terrain.rename(['elevation', 'slope', 'aspect', 'hillshade'])
        
        task = ee.batch.Export.image.toDrive(
            image=terrain.clip(self.roi),
            description='terrain_indices',
            folder='GRACE_downscaling',
            scale=30,  # 30m resolution
            crs='EPSG:4326',
            region=self.roi,
            maxPixels=1e13
        )
        task.start()
        return task

    def download_modis_products(self):
        """Download monthly MODIS products"""
        tasks = []
        
        # Generate dates
        date_pairs = self.generate_date_pairs()
        
        for year, month in date_pairs:
            print(f"Processing MODIS data for {year}-{month:02d}")
            
            start_date = ee.Date.fromYMD(year, month, 1)
            end_date = start_date.advance(1, 'month')
            
            # MODIS Snow Cover
            snow = ee.ImageCollection("MODIS/006/MOD10A1") \
                .select('NDSI_Snow_Cover') \
                .filterDate(start_date, end_date) \
                .mean()
            
            # MODIS VI (EVI and NDVI)
            vi = ee.ImageCollection("MODIS/006/MOD13Q1") \
                .select(['EVI', 'NDVI']) \
                .filterDate(start_date, end_date) \
                .mean()
            
            # MODIS LAI/FPAR
            lai = ee.ImageCollection("MODIS/006/MOD15A2H") \
                .select(['Lai_500m', 'Fpar_500m']) \
                .filterDate(start_date, end_date) \
                .mean()
            
            # MODIS LST
            lst = ee.ImageCollection("MODIS/006/MOD11A1") \
                .select(['LST_Day_1km', 'LST_Night_1km']) \
                .filterDate(start_date, end_date) \
                .mean()
            
            # Export tasks
            tasks.extend([
                ee.batch.Export.image.toDrive(
                    image=snow.clip(self.roi),
                    description=f'snow_cover_{year}_{month:02d}',
                    folder='GRACE_downscaling',
                    scale=500,
                    crs='EPSG:4326',
                    region=self.roi,
                    maxPixels=1e13
                ),
                ee.batch.Export.image.toDrive(
                    image=vi.clip(self.roi),
                    description=f'vegetation_indices_{year}_{month:02d}',
                    folder='GRACE_downscaling',
                    scale=250,
                    crs='EPSG:4326',
                    region=self.roi,
                    maxPixels=1e13
                ),
                ee.batch.Export.image.toDrive(
                    image=lai.clip(self.roi),
                    description=f'lai_fpar_{year}_{month:02d}',
                    folder='GRACE_downscaling',
                    scale=500,
                    crs='EPSG:4326',
                    region=self.roi,
                    maxPixels=1e13
                ),
                ee.batch.Export.image.toDrive(
                    image=lst.clip(self.roi),
                    description=f'lst_{year}_{month:02d}',
                    folder='GRACE_downscaling',
                    scale=1000,
                    crs='EPSG:4326',
                    region=self.roi,
                    maxPixels=1e13
                )
            ])
            
            [task.start() for task in tasks[-4:]]
        
        return tasks

    def download_gldas(self):
        """Download GLDAS soil moisture and other hydrological variables"""
        print("Processing GLDAS data...")
        tasks = []
        
        date_pairs = self.generate_date_pairs()
        
        for year, month in date_pairs:
            print(f"Processing GLDAS for {year}-{month:02d}")
            
            start_date = ee.Date.fromYMD(year, month, 1)
            end_date = start_date.advance(1, 'month')
            
            gldas = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
                .select([
                    'SoilMoi0_10cm_inst',   # Soil moisture 0-10cm
                    'SoilMoi10_40cm_inst',  # Soil moisture 10-40cm
                    'RootMoist_inst',       # Root zone soil moisture
                    'CanopInt_inst',        # Canopy water storage
                    'TWS_inst'              # Total water storage
                ]) \
                .filterDate(start_date, end_date) \
                .mean()
            
            task = ee.batch.Export.image.toDrive(
                image=gldas.clip(self.roi),
                description=f'gldas_{year}_{month:02d}',
                folder='GRACE_downscaling',
                scale=25000,  # GLDAS resolution
                crs='EPSG:4326',
                region=self.roi,
                maxPixels=1e13
            )
            task.start()
            tasks.append(task)
            
        return tasks

    def generate_date_pairs(self):
        """Generate list of year-month pairs"""
        start = datetime.strptime(self.start_date, '%Y-%m-%d')
        end = datetime.strptime(self.end_date, '%Y-%m-%d')
        
        date_pairs = []
        current = start
        
        while current <= end:
            date_pairs.append((current.year, current.month))
            if current.month == 12:
                current = datetime(current.year + 1, 1, 1)
            else:
                current = datetime(current.year, current.month + 1, 1)
        
        return date_pairs

    def download_all(self):
        """Download all additional datasets"""
        tasks = []
        
        # Download terrain data
        print("Starting terrain data download...")
        tasks.append(self.download_terrain_data())
        
        # Download MODIS products
        print("Starting MODIS products download...")
        tasks.extend(self.download_modis_products())
        
        # Download GLDAS data
        print("Starting GLDAS data download...")
        tasks.extend(self.download_gldas())
        
        return tasks

def main():
    # Study area bounds [xmin, ymin, xmax, ymax]
    bounds = [84.89, 26.47, 86.96, 27.19]
    
    collector = AdditionalDataCollector(bounds)
    tasks = collector.download_all()
    
    print(f"\nInitiated {len(tasks)} export tasks.")
    print("Files will be exported to 'GRACE_downscaling' folder in your Google Drive.")
    print("Note: The export process may take some time to complete.")

if __name__ == "__main__":
    main()

Starting terrain data download...
Processing terrain data...
Starting MODIS products download...
Processing MODIS data for 2002-01


c:\Users\J01013381\AppData\Local\miniconda3\envs\gmap\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD10A1! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD10A1

  warnings.warn(warning, category=DeprecationWarning)
c:\Users\J01013381\AppData\Local\miniconda3\envs\gmap\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD13Q1! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13Q1

  warnings.warn(warning, category=DeprecationWarning)
c:\Users\J01013381\AppData\Local\miniconda3\envs\gmap\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD15A2H! You are using a deprecated asset.
To ensure continued func

Processing MODIS data for 2002-02
Processing MODIS data for 2002-03
Processing MODIS data for 2002-04
Processing MODIS data for 2002-05
Processing MODIS data for 2002-06
Processing MODIS data for 2002-07
Processing MODIS data for 2002-08
Processing MODIS data for 2002-09
Processing MODIS data for 2002-10
Processing MODIS data for 2002-11
Processing MODIS data for 2002-12
Processing MODIS data for 2003-01
Processing MODIS data for 2003-02
Processing MODIS data for 2003-03
Processing MODIS data for 2003-04
Processing MODIS data for 2003-05
Processing MODIS data for 2003-06
Processing MODIS data for 2003-07
Processing MODIS data for 2003-08
Processing MODIS data for 2003-09
Processing MODIS data for 2003-10
Processing MODIS data for 2003-11
Processing MODIS data for 2003-12
Processing MODIS data for 2004-01
Processing MODIS data for 2004-02
Processing MODIS data for 2004-03
Processing MODIS data for 2004-04
Processing MODIS data for 2004-05
Processing MODIS data for 2004-06
Processing MOD

In [15]:
import ee
import geemap
from datetime import datetime
import os

ee.Initialize(project = 'ee-074bce151saurav')

class AdditionalDataCollector:
    def __init__(self, study_area_bounds):
        """Initialize GEE and set up study area"""
        ee.Initialize()
        self.roi = ee.Geometry.Rectangle(study_area_bounds)
        self.start_date = '2002-01-01'
        self.end_date = '2013-12-31'

    def download_terrain_data(self):
        """Download ALOS DEM and calculate terrain indices"""
        print("Processing terrain data...")
        
        # Get ALOS DEM
        dem = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").select('DSM').mean()
        
        # Calculate terrain indices
        slope = ee.Terrain.slope(dem)
        aspect = ee.Terrain.aspect(dem)
        hillshade = ee.Terrain.hillshade(dem)
        
        # Combine all terrain features
        terrain = dem.addBands(slope).addBands(aspect).addBands(hillshade)
        terrain = terrain.rename(['elevation', 'slope', 'aspect', 'hillshade'])
        
        task = ee.batch.Export.image.toDrive(
            image=terrain.clip(self.roi),
            description='terrain_indices',
            folder='GRACE_downscaling',
            scale=30,  # 30m resolution
            crs='EPSG:4326',
            region=self.roi,
            maxPixels=1e13
        )
        task.start()
        return task

    def download_modis_products(self):
        """Download monthly MODIS products with updated collections"""
        tasks = []
        
        # Generate dates
        date_pairs = self.generate_date_pairs()
        
        for year, month in date_pairs:
            print(f"Processing MODIS data for {year}-{month:02d}")
            
            start_date = ee.Date.fromYMD(year, month, 1)
            end_date = start_date.advance(1, 'month')
            
            # MODIS Snow Cover (Collection 61)
            snow = ee.ImageCollection("MODIS/061/MOD10A1") \
                .select('NDSI_Snow_Cover') \
                .filterDate(start_date, end_date) \
                .mean()
            
            # MODIS VI (Collection 61)
            vi = ee.ImageCollection("MODIS/061/MOD13Q1") \
                .select(['EVI', 'NDVI']) \
                .filterDate(start_date, end_date) \
                .mean()
            
            # MODIS LAI/FPAR (Collection 61)
            lai = ee.ImageCollection("MODIS/061/MOD15A2H") \
                .select(['Lai', 'Fpar']) \
                .filterDate(start_date, end_date) \
                .mean()
            
            # MODIS LST (Collection 61)
            lst = ee.ImageCollection("MODIS/061/MOD11A1") \
                .select(['LST_Day_1km', 'LST_Night_1km']) \
                .filterDate(start_date, end_date) \
                .mean()
            
            # Export tasks
            tasks.extend([
                ee.batch.Export.image.toDrive(
                    image=snow.clip(self.roi),
                    description=f'snow_cover_{year}_{month:02d}',
                    folder='GRACE_downscaling',
                    scale=500,
                    crs='EPSG:4326',
                    region=self.roi,
                    maxPixels=1e13
                ),
                ee.batch.Export.image.toDrive(
                    image=vi.clip(self.roi),
                    description=f'vegetation_indices_{year}_{month:02d}',
                    folder='GRACE_downscaling',
                    scale=250,
                    crs='EPSG:4326',
                    region=self.roi,
                    maxPixels=1e13
                ),
                ee.batch.Export.image.toDrive(
                    image=lai.clip(self.roi),
                    description=f'lai_fpar_{year}_{month:02d}',
                    folder='GRACE_downscaling',
                    scale=500,
                    crs='EPSG:4326',
                    region=self.roi,
                    maxPixels=1e13
                ),
                ee.batch.Export.image.toDrive(
                    image=lst.clip(self.roi),
                    description=f'lst_{year}_{month:02d}',
                    folder='GRACE_downscaling',
                    scale=1000,
                    crs='EPSG:4326',
                    region=self.roi,
                    maxPixels=1e13
                )
            ])
            
            [task.start() for task in tasks[-4:]]
        
        return tasks

    def download_gldas(self):
        """Download GLDAS soil moisture and other hydrological variables"""
        print("Processing GLDAS data...")
        tasks = []
        
        date_pairs = self.generate_date_pairs()
        
        for year, month in date_pairs:
            print(f"Processing GLDAS for {year}-{month:02d}")
            
            start_date = ee.Date.fromYMD(year, month, 1)
            end_date = start_date.advance(1, 'month')
            
            gldas = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
                .select([
                    'SoilMoi0_10cm_inst',   # Soil moisture 0-10cm
                    'SoilMoi10_40cm_inst',  # Soil moisture 10-40cm
                    'RootMoist_inst',       # Root zone soil moisture
                    'CanopInt_inst',        # Canopy water storage
                    'TWS_inst'              # Total water storage
                ]) \
                .filterDate(start_date, end_date) \
                .mean()
            
            task = ee.batch.Export.image.toDrive(
                image=gldas.clip(self.roi),
                description=f'gldas_{year}_{month:02d}',
                folder='GRACE_downscaling',
                scale=25000,  # GLDAS resolution
                crs='EPSG:4326',
                region=self.roi,
                maxPixels=1e13
            )
            task.start()
            tasks.append(task)
            
        return tasks

    def generate_date_pairs(self):
        """Generate list of year-month pairs"""
        start = datetime.strptime(self.start_date, '%Y-%m-%d')
        end = datetime.strptime(self.end_date, '%Y-%m-%d')
        
        date_pairs = []
        current = start
        
        while current <= end:
            date_pairs.append((current.year, current.month))
            if current.month == 12:
                current = datetime(current.year + 1, 1, 1)
            else:
                current = datetime(current.year, current.month + 1, 1)
        
        return date_pairs

    def download_all(self):
        """Download all additional datasets"""
        tasks = []
        
        # Download terrain data
        print("Starting terrain data download...")
        tasks.append(self.download_terrain_data())
        
        # Download MODIS products
        print("Starting MODIS products download...")
        tasks.extend(self.download_modis_products())
        
        # Download GLDAS data
        print("Starting GLDAS data download...")
        tasks.extend(self.download_gldas())
        
        return tasks

def main():
    # Study area bounds [xmin, ymin, xmax, ymax]
    bounds = [84.89, 26.47, 86.96, 27.19]
    
    collector = AdditionalDataCollector(bounds)
    tasks = collector.download_all()
    
    print(f"\nInitiated {len(tasks)} export tasks.")
    print("Files will be exported to 'GRACE_downscaling' folder in your Google Drive.")
    print("Note: The export process may take some time to complete.")

if __name__ == "__main__":
    main()

Starting terrain data download...
Processing terrain data...
Starting MODIS products download...
Processing MODIS data for 2002-01
Processing MODIS data for 2002-02
Processing MODIS data for 2002-03
Processing MODIS data for 2002-04
Processing MODIS data for 2002-05
Processing MODIS data for 2002-06
Processing MODIS data for 2002-07
Processing MODIS data for 2002-08
Processing MODIS data for 2002-09
Processing MODIS data for 2002-10
Processing MODIS data for 2002-11
Processing MODIS data for 2002-12
Processing MODIS data for 2003-01
Processing MODIS data for 2003-02
Processing MODIS data for 2003-03
Processing MODIS data for 2003-04
Processing MODIS data for 2003-05
Processing MODIS data for 2003-06
Processing MODIS data for 2003-07
Processing MODIS data for 2003-08
Processing MODIS data for 2003-09
Processing MODIS data for 2003-10
Processing MODIS data for 2003-11
Processing MODIS data for 2003-12
Processing MODIS data for 2004-01
Processing MODIS data for 2004-02
Processing MODIS da